##  mainからデータを取得できるようにする

In [1]:
import requests
import sqlite3

# URLからJSONデータを取得
url = "http://www.jma.go.jp/bosai/common/const/area.json"
response = requests.get(url)

# レスポンスのエンコーディングをUTF-8に設定
response.encoding = 'utf-8'

# JSONデータをPythonの辞書として解析
area_json = response.json()

# "centers" キー内のデータを取得
centers = area_json.get("centers", {})

# SQLiteデータベース接続
db_filename = 'area_info.db'
conn = sqlite3.connect(db_filename)
cursor = conn.cursor()

# テーブルの作成（もしテーブルが存在しない場合）
cursor.execute("""
CREATE TABLE IF NOT EXISTS area_info (
    id INTEGER PRIMARY KEY,
    area_code TEXT,
    area_name TEXT,
    en_name TEXT,
    office_name TEXT,
    children_code TEXT
)
""")

# idの初期化
id_counter = 1

# 各エリア情報をSQLiteデータベースに挿入
for area_code, area_info in centers.items():
    name = area_info.get("name", "N/A")
    enName = area_info.get("enName", "N/A")
    officeName = area_info.get("officeName", "N/A")
    children = area_info.get("children", [])

    # 親エリアの情報をSQLiteに書き込む
    if children:
        for region_code in children:
            # 子エリアIDを挿入
            cursor.execute("""
            INSERT INTO area_info (id, area_code, area_name, en_name, office_name, children_code)
            VALUES (?, ?, ?, ?, ?, ?)
            """, (id_counter, area_code, name, enName, officeName, region_code))
            id_counter += 1
    else:
        # 子エリアがいない場合
        cursor.execute("""
        INSERT INTO area_info (id, area_code, area_name, en_name, office_name, children_code)
        VALUES (?, ?, ?, ?, ?, ?)
        """, (id_counter, area_code, name, enName, officeName, 'N/A'))
        id_counter += 1

# コミットして変更を保存
conn.commit()

# データベース接続を閉じる
conn.close()

IntegrityError: UNIQUE constraint failed: area_info.id

In [2]:
import sqlite3

# データベースのパスと名前
db_filename = 'area_info.db'

# DBに接続
con = sqlite3.connect(db_filename)

# SQLを実行するためのカーソルオブジェクトを取得
cur = con.cursor()

# データを参照するSQL文
# 例: area_infoテーブルのすべてのデータを取得する
sql_select = "SELECT * FROM area_info;"

# SQLを実行
cur.execute(sql_select)

# データベースから取得したデータをループで表示
for row in cur:
    # 行データ(row)はタプルなので、アンパックして列データを取得
    id, area_code, area_name, en_name, office_name, children_code = row
    print(f"id: {id}, area_code: {area_code}, area_name: {area_name}, en_name: {en_name}, office_name: {office_name}, children_code: {children_code}")

# DBへの接続を閉じる
con.close()


id: 1, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 011000
id: 2, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 012000
id: 3, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 013000
id: 4, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 014030
id: 5, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 014100
id: 6, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 015000
id: 7, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 016000
id: 8, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 017000
id: 9, area_code: 010200, area_name: 東北地方, en_name: Tohoku, office_name: 仙台管区気象台, children_code: 020000
id: 10, area_code: 010200, area_name: 東北

##  市町村それぞれの天気予報を取得する

In [3]:
import sqlite3
import requests
import csv

# データベースのパスと名前
db_filename = 'area_info.db'

# DBに接続
con = sqlite3.connect(db_filename)

# SQLを実行するためのカーソルオブジェクトを取得
cur = con.cursor()

# データを参照するSQL文
sql_select = "SELECT * FROM area_info;"

# CSVファイルを開く
with open('weather_forecast.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # CSVヘッダーを書き込む（id, children_code を追加）
    writer.writerow(["ID", "Children Code", "Area Name", "Area Code", "Date", "Weather Codes", "Weathers", "Winds", "Waves"])

    # SQLを実行
    cur.execute(sql_select)

    # データベースから取得したデータをループで処理
    for row in cur:
        # 行データ(row)はタプルなので、アンパックして列データを取得
        id, area_code, area_name, en_name, office_name, children_code = row
        print(f"id: {id}, area_code: {area_code}, area_name: {area_name}, en_name: {en_name}, office_name: {office_name}, children_code: {children_code}")

        # children_codeを使ってURLを作成
        url = f"https://www.jma.go.jp/bosai/forecast/data/forecast/{children_code}.json"
        
        try:
            # URLからJSONデータを取得
            response = requests.get(url)
            response.raise_for_status()  # レスポンスが成功でない場合、例外を発生

            # JSONデータを解析
            data = response.json()

            # 時刻（timeDefines）の情報を取得
            time_defines = data[0]['timeSeries'][0]['timeDefines']
            
            # 各エリアの情報を取得
            areas = data[0]['timeSeries'][0]['areas']  # ここでエリアのリストを取得

            # 各エリアの情報をループ処理
            for area in areas:
                area_name = area['area']['name']
                area_code = area['area']['code']
                weather_codes = area.get('weatherCodes', ['なし'])
                weathers = area.get('weathers', ['なし'])
                winds = area.get('winds', ['なし'])
                waves = area.get('waves', ['なし'])

                # データを1行ずつ書き込む
                for i in range(len(time_defines)):
                    row = [
                        id,  # IDを最初に追加
                        children_code,  # Children Codeを追加
                        area_name,  # Area Nameを追加
                        area_code,  # Area Codeを追加
                        time_defines[i],  # Date
                        weather_codes[i] if i < len(weather_codes) else 'なし',  # Weather Codes
                        weathers[i] if i < len(weathers) else 'なし',  # Weathers
                        winds[i] if i < len(winds) else 'なし',  # Winds
                        waves[i] if i < len(waves) else 'なし'  # Waves
                    ]
                    # CSVファイルに書き込む
                    writer.writerow(row)

        except requests.exceptions.RequestException as e:
            # リクエストエラーが発生した場合
            print(f"Error fetching data from URL {url}: {e}")

# DBへの接続を閉じる
con.close()


id: 1, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 011000
id: 2, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 012000
id: 3, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 013000
id: 4, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 014030
Error fetching data from URL https://www.jma.go.jp/bosai/forecast/data/forecast/014030.json: 404 Client Error: Not Found for url: https://www.jma.go.jp/bosai/forecast/data/forecast/014030.json
id: 5, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 014100
id: 6, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 015000
id: 7, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 016000
id: 8, area_code: 010100, area_name: 北海道地方, en_name: Hokka

## テーブルの中にさらにテーブルを作成する

In [5]:
import sqlite3
import requests
import csv

# データベースのパスと名前
db_filename = 'area_info.db'

# DBに接続
con = sqlite3.connect(db_filename)

# SQLを実行するためのカーソルオブジェクトを取得
cur = con.cursor()

# データを参照するSQL文（テーブル名をareas_weather_infoに変更）
sql_select = "SELECT * FROM areas_weather_info;"

# CSVファイルを開く
with open('weather_forecast.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # CSVヘッダーを書き込む（id, children_code を追加）
    writer.writerow(["ID", "Children Code", "Area Name", "Area Code", "Date", "Weather Codes", "Weathers", "Winds", "Waves"])

    # SQLを実行
    cur.execute(sql_select)

    # データベースから取得したデータをループで処理
    for row in cur:
        # 行データ(row)はタプルなので、アンパックして列データを取得
        id, area_code, area_name, en_name, office_name, children_code = row
        print(f"id: {id}, area_code: {area_code}, area_name: {area_name}, en_name: {en_name}, office_name: {office_name}, children_code: {children_code}")

        # children_codeを使ってURLを作成
        url = f"https://www.jma.go.jp/bosai/forecast/data/forecast/{children_code}.json"
        
        try:
            # URLからJSONデータを取得
            response = requests.get(url)
            response.raise_for_status()  # レスポンスが成功でない場合、例外を発生

            # JSONデータを解析
            data = response.json()

            # 時刻（timeDefines）の情報を取得
            time_defines = data[0]['timeSeries'][0]['timeDefines']
            
            # 各エリアの情報を取得
            areas = data[0]['timeSeries'][0]['areas']  # ここでエリアのリストを取得

            # 各エリアの情報をループ処理
            for area in areas:
                area_name = area['area']['name']
                area_code = area['area']['code']
                weather_codes = area.get('weatherCodes', ['なし'])
                weathers = area.get('weathers', ['なし'])
                winds = area.get('winds', ['なし'])
                waves = area.get('waves', ['なし'])

                # データを1行ずつ書き込む
                for i in range(len(time_defines)):
                    row = [
                        id,  # IDを最初に追加
                        children_code,  # Children Codeを追加
                        area_name,  # Area Nameを追加
                        area_code,  # Area Codeを追加
                        time_defines[i],  # Date
                        weather_codes[i] if i < len(weather_codes) else 'なし',  # Weather Codes
                        weathers[i] if i < len(weathers) else 'なし',  # Weathers
                        winds[i] if i < len(winds) else 'なし',  # Winds
                        waves[i] if i < len(waves) else 'なし'  # Waves
                    ]
                    # CSVファイルに書き込む
                    writer.writerow(row)

        except requests.exceptions.RequestException as e:
            # リクエストエラーが発生した場合
            print(f"Error fetching data from URL {url}: {e}")

# DBへの接続を閉じる
con.close()


OperationalError: no such table: areas_weather_info

In [7]:
import sqlite3
import requests
import csv

# データベースのパスと名前
db_filename = 'area_info.db'

# DBに接続
con = sqlite3.connect(db_filename)

# SQLを実行するためのカーソルオブジェクトを取得
cur = con.cursor()

# データを参照するSQL文
sql_select = "SELECT * FROM area_info;"

# CSVファイルを開く
with open('weather_forecast.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # CSVヘッダーを書き込む（id, children_code を追加）
    writer.writerow(["ID", "Children Code", "Area Name", "Area Code", "Date", "Weather Codes", "Weathers", "Winds", "Waves"])

    # SQLを実行
    cur.execute(sql_select)

    # データベースから取得したデータをループで処理
    for row in cur:
        # 行データ(row)はタプルなので、アンパックして列データを取得
        id, area_code, area_name, en_name, office_name, children_code = row
        print(f"id: {id}, area_code: {area_code}, area_name: {area_name}, en_name: {en_name}, office_name: {office_name}, children_code: {children_code}")

        # children_codeを使ってURLを作成
        url = f"https://www.jma.go.jp/bosai/forecast/data/forecast/{children_code}.json"
        
        try:
            # URLからJSONデータを取得
            response = requests.get(url)
            response.raise_for_status()  # レスポンスが成功でない場合、例外を発生

            # JSONデータを解析
            data = response.json()

            # 時刻（timeDefines）の情報を取得
            time_defines = data[0]['timeSeries'][0]['timeDefines']
            
            # 各エリアの情報を取得
            areas = data[0]['timeSeries'][0]['areas']  # ここでエリアのリストを取得

            # 各エリアの情報をループ処理
            for area in areas:
                area_name = area['area']['name']
                area_code = area['area']['code']
                weather_codes = area.get('weatherCodes', ['なし'])
                weathers = area.get('weathers', ['なし'])
                winds = area.get('winds', ['なし'])
                waves = area.get('waves', ['なし'])

                # データを1行ずつ書き込む
                for i in range(len(time_defines)):
                    row = [
                        id,  # IDを最初に追加
                        children_code,  # Children Codeを追加
                        area_name,  # Area Nameを追加
                        area_code,  # Area Codeを追加
                        time_defines[i],  # Date
                        weather_codes[i] if i < len(weather_codes) else 'なし',  # Weather Codes
                        weathers[i] if i < len(weathers) else 'なし',  # Weathers
                        winds[i] if i < len(winds) else 'なし',  # Winds
                        waves[i] if i < len(waves) else 'なし'  # Waves
                    ]
                    # CSVファイルに書き込む
                    writer.writerow(row)

        except requests.exceptions.RequestException as e:
            # リクエストエラーが発生した場合
            print(f"Error fetching data from URL {url}: {e}")

# DBへの接続を閉じる
con.close()

id: 1, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 011000
id: 2, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 012000
id: 3, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 013000
id: 4, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 014030
Error fetching data from URL https://www.jma.go.jp/bosai/forecast/data/forecast/014030.json: 404 Client Error: Not Found for url: https://www.jma.go.jp/bosai/forecast/data/forecast/014030.json
id: 5, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 014100
id: 6, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 015000
id: 7, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 016000
id: 8, area_code: 010100, area_name: 北海道地方, en_name: Hokka

In [2]:
import sqlite3
import requests

# データベースのパスと名前
db_filename = 'weather_forecast.db'

# 新しいデータベースに接続（存在しない場合は作成されます）
con = sqlite3.connect(db_filename)
cur = con.cursor()

# 新しいテーブルを作成するSQL文
cur.execute("""
CREATE TABLE IF NOT EXISTS forecast_data (
    id INTEGER,
    children_code TEXT,
    area_name TEXT,
    area_code TEXT,
    date TEXT,
    weather_codes TEXT,
    weathers TEXT,
    winds TEXT,
    waves TEXT
)
""")

# 既存のarea_info.dbからデータを取得
area_info_db = 'area_info.db'
con_area_info = sqlite3.connect(area_info_db)
cur_area_info = con_area_info.cursor()

# SQLを実行して、area_infoテーブルからデータを取得
sql_select = "SELECT * FROM area_info;"
cur_area_info.execute(sql_select)

# データベースから取得したデータをループで処理
for row in cur_area_info:
    # 行データ(row)はタプルなので、アンパックして列データを取得
    id, area_code, area_name, en_name, office_name, children_code = row
    print(f"id: {id}, area_code: {area_code}, area_name: {area_name}, en_name: {en_name}, office_name: {office_name}, children_code: {children_code}")

    # children_codeを使ってURLを作成
    url = f"https://www.jma.go.jp/bosai/forecast/data/forecast/{children_code}.json"
    
    try:
        # URLからJSONデータを取得
        response = requests.get(url)
        response.raise_for_status()  # レスポンスが成功でない場合、例外を発生

        # JSONデータを解析
        data = response.json()

        # 時刻（timeDefines）の情報を取得
        time_defines = data[0]['timeSeries'][0]['timeDefines']
        
        # 各エリアの情報を取得
        areas = data[0]['timeSeries'][0]['areas']  # ここでエリアのリストを取得

        # 各エリアの情報をループ処理
        for area in areas:
            area_name = area['area']['name']
            area_code = area['area']['code']
            weather_codes = area.get('weatherCodes', ['なし'])
            weathers = area.get('weathers', ['なし'])
            winds = area.get('winds', ['なし'])
            waves = area.get('waves', ['なし'])

            # データを1行ずつDBに挿入
            for i in range(len(time_defines)):
                # 行データを準備
                row_data = (
                    id,  # ID
                    children_code,  # Children Code
                    area_name,  # Area Name
                    area_code,  # Area Code
                    time_defines[i],  # Date
                    weather_codes[i] if i < len(weather_codes) else 'なし',  # Weather Codes
                    weathers[i] if i < len(weathers) else 'なし',  # Weathers
                    winds[i] if i < len(winds) else 'なし',  # Winds
                    waves[i] if i < len(waves) else 'なし'  # Waves
                )
                # DBにデータを挿入
                cur.execute("""
                INSERT INTO forecast_data (
                    id, children_code, area_name, area_code, date, weather_codes, weathers, winds, waves
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                """, row_data)

    except requests.exceptions.RequestException as e:
        # リクエストエラーが発生した場合
        print(f"Error fetching data from URL {url}: {e}")

# コミットして変更を保存
con.commit()

# DBへの接続を閉じる
con_area_info.close()
con.close()


id: 1, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 011000
id: 2, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 012000
id: 3, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 013000
id: 4, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 014030
Error fetching data from URL https://www.jma.go.jp/bosai/forecast/data/forecast/014030.json: 404 Client Error: Not Found for url: https://www.jma.go.jp/bosai/forecast/data/forecast/014030.json
id: 5, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 014100
id: 6, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 015000
id: 7, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 016000
id: 8, area_code: 010100, area_name: 北海道地方, en_name: Hokka

##  CSVごとのデータを取得

In [14]:
import sqlite3
import requests
import csv

# データベースのパスと名前
db_filename = 'area_info.db'

# DBに接続
con = sqlite3.connect(db_filename)

# SQLを実行するためのカーソルオブジェクトを取得
cur = con.cursor()

# データを参照するSQL文
sql_select = "SELECT * FROM area_info;"

# CSVファイルを開く
with open('weather_forecast.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # CSVヘッダーを書き込む（id, children_code を追加）
    writer.writerow(["ID", "Children Code", "Area Name", "Area Code", "Date", "Weather Codes", "Weathers", "Winds", "Waves"])

    # SQLを実行
    cur.execute(sql_select)

    # データベースから取得したデータをループで処理
    for row in cur:
        # 行データ(row)はタプルなので、アンパックして列データを取得
        id, area_code, area_name, en_name, office_name, children_code = row
        print(f"id: {id}, area_code: {area_code}, area_name: {area_name}, en_name: {en_name}, office_name: {office_name}, children_code: {children_code}")

        # children_codeを使ってURLを作成
        url = f"https://www.jma.go.jp/bosai/forecast/data/forecast/{children_code}.json"
        
        try:
            # URLからJSONデータを取得
            response = requests.get(url)
            response.raise_for_status()  # レスポンスが成功でない場合、例外を発生

            # JSONデータを解析
            data = response.json()

            # 時刻（timeDefines）の情報を取得
            time_defines = data[0]['timeSeries'][0]['timeDefines']
            
            # 各エリアの情報を取得
            areas = data[0]['timeSeries'][0]['areas']  # ここでエリアのリストを取得

            # 各エリアの情報をループ処理
            for area in areas:
                area_name = area['area']['name']
                area_code = area['area']['code']
                weather_codes = area.get('weatherCodes', ['なし'])
                weathers = area.get('weathers', ['なし'])
                winds = area.get('winds', ['なし'])
                waves = area.get('waves', ['なし'])

                # データを1行ずつ書き込む
                for i in range(len(time_defines)):
                    row = [
                        id,  # IDを最初に追加
                        children_code,  # Children Codeを追加
                        area_name,  # Area Nameを追加
                        area_code,  # Area Codeを追加
                        time_defines[i],  # Date
                        weather_codes[i] if i < len(weather_codes) else 'なし',  # Weather Codes
                        weathers[i] if i < len(weathers) else 'なし',  # Weathers
                        winds[i] if i < len(winds) else 'なし',  # Winds
                        waves[i] if i < len(waves) else 'なし'  # Waves
                    ]
                    # CSVファイルに書き込む
                    writer.writerow(row)

        except requests.exceptions.RequestException as e:
            # リクエストエラーが発生した場合
            print(f"Error fetching data from URL {url}: {e}")

# DBへの接続を閉じる
con.close()

id: 1, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 011000
id: 2, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 012000
id: 3, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 013000
id: 4, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 014030
Error fetching data from URL https://www.jma.go.jp/bosai/forecast/data/forecast/014030.json: 404 Client Error: Not Found for url: https://www.jma.go.jp/bosai/forecast/data/forecast/014030.json
id: 5, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 014100
id: 6, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 015000
id: 7, area_code: 010100, area_name: 北海道地方, en_name: Hokkaido, office_name: 札幌管区気象台, children_code: 016000
id: 8, area_code: 010100, area_name: 北海道地方, en_name: Hokka

##  DBのデータを取得

In [3]:
import sqlite3
import requests

# データベースのパスと名前
db_filename = 'detail_areas_weather.db'

# DBに接続
con = sqlite3.connect(db_filename)

# SQLを実行するためのカーソルオブジェクトを取得
cur = con.cursor()

# データを参照するSQL文
sql_select = "SELECT * FROM area_info;"

# general_info テーブルを作成（地域情報を格納）
cur.execute('''
    CREATE TABLE IF NOT EXISTS general_info (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        area_code TEXT NOT NULL,
        area_name TEXT NOT NULL,
        en_name TEXT,
        office_name TEXT,
        children_code TEXT NOT NULL,
        UNIQUE(area_code, children_code)
    );
''')
con.commit()

# SQLを実行して、area_infoからデータを取得
cur.execute(sql_select)

# データベースから取得したデータをループで処理
for row in cur:
    # 行データ(row)はタプルなので、アンパックして列データを取得
    id, area_code, area_name, en_name, office_name, children_code = row
    print(f"id: {id}, area_code: {area_code}, area_name: {area_name}, en_name: {en_name}, office_name: {office_name}, children_code: {children_code}")

    # general_info テーブルにデータを挿入
    cur.execute('''
        INSERT OR REPLACE INTO general_info (area_code, area_name, en_name, office_name, children_code)
        VALUES (?, ?, ?, ?, ?)
    ''', (area_code, area_name, en_name, office_name, children_code))
    con.commit()

    # children_codeごとの weather_forecast テーブルを作成
    table_name = f"weather_forecast_{children_code}"
    create_table_sql = f'''
        CREATE TABLE IF NOT EXISTS {table_name} (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            date TEXT NOT NULL,
            weather_codes TEXT NOT NULL,
            weathers TEXT NOT NULL,
            winds TEXT NOT NULL,
            waves TEXT NOT NULL
        );
    '''
    cur.execute(create_table_sql)
    con.commit()

    # children_codeを使ってURLを作成
    url = f"https://www.jma.go.jp/bosai/forecast/data/forecast/{children_code}.json"
    
    try:
        # URLからJSONデータを取得
        response = requests.get(url)
        response.raise_for_status()  # レスポンスが成功でない場合、例外を発生

        # JSONデータを解析
        data = response.json()

        # 時刻（timeDefines）の情報を取得
        time_defines = data[0]['timeSeries'][0]['timeDefines']
        
        # 各エリアの情報を取得
        areas = data[0]['timeSeries'][0]['areas']  # ここでエリアのリストを取得

        # 各エリアの情報をループ処理
        for area in areas:
            area_name = area['area']['name']
            area_code = area['area']['code']
            weather_codes = area.get('weatherCodes', ['なし'])
            weathers = area.get('weathers', ['なし'])
            winds = area.get('winds', ['なし'])
            waves = area.get('waves', ['なし'])

            # データを1行ずつ書き込む
            for i in range(len(time_defines)):
                row = [
                    id,  # IDを最初に追加
                    children_code,  # Children Codeを追加
                    area_name,  # Area Nameを追加
                    area_code,  # Area Codeを追加
                    time_defines[i],  # Date
                    weather_codes[i] if i < len(weather_codes) else 'なし',  # Weather Codes
                    weathers[i] if i < len(weathers) else 'なし',  # Weathers
                    winds[i] if i < len(winds) else 'なし',  # Winds
                    waves[i] if i < len(waves) else 'なし'  # Waves
                ]
                # weather_forecast_<children_code> テーブルにデータを挿入
                insert_sql = f'''
                    INSERT INTO {table_name} (date, weather_codes, weathers, winds, waves)
                    VALUES (?, ?, ?, ?, ?)
                '''
                cur.execute(insert_sql, (
                    time_defines[i],
                    weather_codes[i] if i < len(weather_codes) else 'なし',
                    weathers[i] if i < len(weathers) else 'なし',
                    winds[i] if i < len(winds) else 'なし',
                    waves[i] if i < len(waves) else 'なし'
                ))
                con.commit()

    except requests.exceptions.RequestException as e:
        # リクエストエラーが発生した場合
        print(f"Error fetching data from URL {url}: {e}")

# DBへの接続を閉じる
con.close()


OperationalError: no such table: area_info

In [ ]:
##